In [1]:
import numpy as np
import pandas as pd
from itertools import product

In [2]:
def load_data(file:str):
    raw = open(file, 'r').read()
    raw = raw.split('\n')
    data = []
    for r in raw:
        data.append([int(x) for x in r])
    return pd.DataFrame(data)
t = load_data('day_9_test.txt')
data = load_data('day_9_input.txt')

In [16]:
tmp = data
tmp.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,3,5,6,7,8,9,1,9,8,7,...,7,6,7,5,9,7,8,9,2,1
1,2,4,5,8,9,1,0,9,7,6,...,5,4,3,4,5,6,9,5,3,0
2,0,1,9,9,8,9,1,9,8,7,...,4,3,2,3,6,7,8,9,9,1
3,9,9,8,7,6,8,9,9,9,7,...,5,5,1,2,3,5,6,7,8,9
4,8,8,7,6,5,9,9,8,9,8,...,7,4,3,4,6,6,7,9,9,0


In [17]:
def get_surrounding_positions(r,c: int):
    return [(r-1,c), (r+1, c), (r, c-1), (r, c+1)]
    
def get_correct_positions(df: pd.DataFrame, positions: list):
    for x in positions:
        if x[0] < 0 or x[1] < 0:
            positions.remove(x)
    
    for x in positions:
        if x[0] >= tmp.shape[0]:
            positions.remove(x)
        elif x[1] >= tmp.shape[1]:
            positions.remove(x)
    return positions

def get_df_values(df, positions: list):
    return [df.iloc[x] for x in positions]

In [18]:
max_ = tmp.shape
values = []
low_points = []
for r,c in product(list(range(0,max_[0])), list(range(0,max_[1]))):
    pos = get_surrounding_positions(r,c)
    corr_pos = get_correct_positions(tmp, pos)
    if all(tmp.iloc[r,c] < x for x in get_df_values(tmp, corr_pos)):
        low_points.append((r,c))
        values.append(tmp.iloc[r,c])
        
values = [x + 1 for x in values]
np.sum(values)

600

# Part 2

In [19]:
def search_horizontally_to_the_right(df: pd.DataFrame, row, start_column: int):
    max_column_idx = df.shape[1] - 1
    data = []
    i = start_column
    while True:
        val = df.iloc[row,i]
        if val < 9:
            data.append([(row,i), val])
        else:
            break
        if i >= max_column_idx:
            break
        else:
            i += 1
    return [x[0] for x in data], [x[1] for x in data]

def search_horizontally_to_the_left(df: pd.DataFrame, row, start_column: int):
    data = []
    if start_column ==0:
        return [], []
    
    i = start_column - 1
    while True:
        val = df.iloc[row,i]
        if val < 9:
            data.append([(row,i), val])
        else:
            break
        if i == 0:
            break
        else:
            i -= 1
    return [x[0] for x in data], [x[1] for x in data]

def search_vertically_to_the_bottom(df: pd.DataFrame, start_row, col: int):
    max_row_idx = df.shape[0] - 1
    data = []
    if start_row == max_row_idx:
        return [], []
    i = start_row + 1
    while True:
        val = df.iloc[i, col]
        if val < 9:
            data.append([(i,col), val])
        else:
            break
        if i >= max_row_idx:
            break
        else:
            i += 1
    return [x[0] for x in data], [x[1] for x in data]
        
def search_vertically_to_the_top(df: pd.DataFrame, start_row, col: int):
    data = []
    if start_row ==0:
        return [], []
    
    i = start_row - 1
    while True:
        val = df.iloc[i,col]
        if val < 9:
            data.append([(i,col), val])
        else:
            break
        if i ==0:
            break
        else:
            i -= 1

    return [x[0] for x in data], [x[1] for x in data]

In [20]:
def position_algorithm(df, r, c):
    h_r_values, h_r = search_horizontally_to_the_right(df, r,c)
    h_l_values, h_l = search_horizontally_to_the_left(df, r,c)
    v_b_values, v_b = search_vertically_to_the_bottom(df, r,c)
    v_t_values, v_t = search_vertically_to_the_top(df, r,c)
    return [h_r_values,h_l_values, v_b_values, v_t_values], [h_r, h_l, v_b, v_t]
position_algorithm(tmp, 4,9)

([[(4, 9), (4, 10), (4, 11)], [], [], [(3, 9), (2, 9), (1, 9), (0, 9)]],
 [[8, 7, 5], [], [], [7, 7, 6, 7]])

In [21]:
def algorithm(df, r, c):
     
    pos, val = position_algorithm(df, r , c)
    h_r_pos = pos[0]; h_l_pos = pos[1]
    h_pos = h_r_pos + h_l_pos
    points_in_neighbourhood = h_pos
    points_in_neighbourhood.sort(key=lambda x: x[1])
    
    for i in set(h_pos):
        v_b_values, _ = search_vertically_to_the_bottom(df, i[0],i[1])
        v_t_values, _ = search_vertically_to_the_top(df, i[0],i[1])
        v_b_values.sort(key=lambda x: x[1])
        v_t_values.sort(key=lambda x: x[1])
        v_pos = v_b_values + v_t_values
        points_in_neighbourhood.extend(v_pos)
    
        for j in set(v_pos):
            h_r_values, _ = search_horizontally_to_the_right(df, j[0],j[1])
            h_l_values, _ = search_horizontally_to_the_left(df, j[0],j[1])
            h_r_values.sort(key=lambda x: x[1])
            h_l_values.sort(key=lambda x: x[1])
            h_pos_tmp = h_r_values + h_l_values
            points_in_neighbourhood.extend(h_pos_tmp)

    points_in_neighbourhood.sort(key=lambda x: x[0])
    points_in_neighbourhood= set(points_in_neighbourhood)
    return len(points_in_neighbourhood)

algorithm(tmp, 2, 4)

48

In [25]:
max_ = tmp.shape
values = []
# for r,c in product(list(range(0,max_[0])), list(range(0,max_[1]))):
for x in low_points:
    r = x[0]
    c = x[1]
    value = algorithm(tmp, r, c)
    values.append(value)
values = sorted(values)
x = 1
for i in values[-3:]:
    x *= i
x

987840